In [0]:
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import csv
import argparse as ap

In [0]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/Computer_Vision_final/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split

In [0]:
#this is the stuff for testing 
csv_path = data_dir + "face_labels.csv"
df = pd.read_csv(csv_path)

In [0]:
df.head()
col_file = list(df.file)
col_file = col_file[:-2]

In [0]:
images = list()
labels = list()
#primary_path = data_dir + "region_maps/"
#sub_dir_list = os.listdir(primary_path)
for i in range(len(col_file)):
  label = int(df['gender'][i])
  holder = col_file[i].split(".")
  name = holder[0] + "_corners.jpg"
  path = data_dir + "corners/" + name
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
  image_data = np.array(img)
  images.append(image_data)
  labels.append(label)
#x = np.array(images)
test_features = np.array(images)
labels = np.array(labels)
#y = np.array(keras.utils.to_categorical(np.array(labels), num_classes=2))
test_labels = np.array(keras.utils.to_categorical(np.array(labels), num_classes=2)) 
#train_features , test_features ,train_labels, test_labels = train_test_split( x , y , test_size=0.4 )

We have just loaded the contour maps in. Now what we want to do is to create a NN

FIRST WE WILL LOAD IN OUR TRAIN DATA>> SORRY IT IS BACKWARD!


In [0]:
#this is the stuff for testing 
csv_path_train = data_dir + "face_train.csv"
df_train = pd.read_csv(csv_path_train)
col_file_train = list(df_train.file)

In [0]:
col_file_train

In [0]:
images_train = list()
labels_train = list()
primary_path = data_dir + "corners/train/"
for i in range(len(col_file_train)):
  label = int(df_train['gender'][i])
  holder = col_file_train[i].split(".")
  name = holder[0] + "_corners.jpg"
  path = data_dir + "corners/train/" + name
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
  image_data = np.array(img)
  
  if image_data.shape == (0,0):
    continue 
  
  images_train.append(image_data)
  labels_train.append(label)

train_features = np.array(images_train)
labels_train = np.array(labels_train)
#y = np.array(keras.utils.to_categorical(np.array(labels_train), num_classes=2))
train_labels = np.array(keras.utils.to_categorical(np.array(labels_train), num_classes=2))
#train_features , test_features ,train_labels, test_labels = train_test_split( x_features , y , test_size=0.4 )

In [0]:
print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)
print(test_labels.shape)

(1615, 128, 128)
(169, 128, 128)
(1615, 2)
(169, 2)


In [0]:
from tensorflow.python.keras import models , optimizers , losses ,activations
from tensorflow.python.keras.layers import *
import tensorflow as tf

In [0]:
class Classifier(object):
    def __init__( self , number_of_classes, dimensions):
        dropout_rate = 0.5
        self.dimensions = dimensions
        input_shape = ( dimensions**2 , )
        convolution_shape = ( dimensions , dimensions , 1 )
        kernel_size = (3 , 3)
        pool_size = (2 , 2)
        strides = 1
        activation_func = activations.relu
        self.layers = [
            Reshape( input_shape=input_shape , target_shape=convolution_shape),
            Conv2D( 32, kernel_size=( 4 , 4 ) , strides=strides , activation=activation_func),
            MaxPooling2D(pool_size=pool_size, strides=strides ),
            Conv2D( 64, kernel_size=( 3 , 3 ) , strides=strides , activation=activation_func),
            MaxPooling2D(pool_size=pool_size , strides=strides),
            Flatten(),
            Dense( 100, activation=activation_func) ,
            Dropout(dropout_rate),
            Dense( number_of_classes, activation=tf.nn.softmax )
        ]
        self.model = tf.keras.Sequential(self.layers)
        self.model.compile(
            optimizer=optimizers.Adam(lr=.001),
            loss=losses.categorical_crossentropy ,
            metrics=['accuracy'] ,
        )
    def fit(self, X, Y, hyperparameters):
        #if you want to not have that epoch printing you can just say verbose  = 0 in the .fit call 
        self.model.fit(X, Y, batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'], shuffle = True)  

    def evaluate(self , test_X , test_Y) :
        return self.model.evaluate(test_X, test_Y)

    def predict(self, X):
        predictions = self.model.predict(X)
        return predictions

    def save_model(self , file_path ):
        self.model.save(file_path )

    def load_model(self , file_path ):
        self.model = models.load_model(file_path)


Now to try and run 

In [0]:
data_dimension = 128
train_features =  train_features.reshape( (  train_features.shape[0] , data_dimension**2  ) ).astype( np.float32 )
test_features= test_features.reshape( ( test_features.shape[0] , data_dimension**2 ) ).astype( np.float32 )
print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)
print(test_labels.shape)

classifier = Classifier( number_of_classes=2, dimensions = 128 )
parameters = {
    'batch_size' : 50 ,
    'epochs' : 10,
}
classifier.fit(train_features , train_labels  , hyperparameters=parameters )

(1615, 16384)
(169, 16384)
(1615, 2)
(169, 2)
Train on 1615 samples
Epoch 1/10
1615/1615 [==============================] - 3s 2ms/sample - loss: 120.4737 - acc: 0.6124
Epoch 2/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.5167 - acc: 0.7659
Epoch 3/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.3080 - acc: 0.8799
Epoch 4/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.1578 - acc: 0.9362
Epoch 5/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.0718 - acc: 0.9740
Epoch 6/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.0435 - acc: 0.9851
Epoch 7/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.0342 - acc: 0.9920
Epoch 8/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.0338 - acc: 0.9901
Epoch 9/10
1615/1615 [==============================] - 2s 1ms/sample - loss: 0.0282 - acc: 0.9944
Epoch 10/10
1615/1615 [================

In [0]:
#classifier.save_model( 'models/model.h5')
preds = classifier.predict(test_features)
loss , accuracy, FP, AUC, TP = classifier.evaluate( test_features , test_labels )
print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ), "False Positive {}".format( FP ), " TruePositive {}".format( TP ), "Area Under Curve{}".format( AUC ) )

169/169 [==============================] - 0s 900us/sample - loss: 1.7010 - acc: 0.5740


ValueError: ignored